In [1]:
import numpy as np
import pandas as pd
from matplotlib import rc
import matplotlib.pyplot as plt
import itertools
import warnings
from matplotlib import rcParams
import scipy.io as sio
rcParams['font.family'] = 'helvetica'
path = 'crop/India_crop/crop_production.csv'
plt.rcParams.update({'font.size': 20})

In [11]:
data = pd.read_csv(path)
data

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0
...,...,...,...,...,...,...,...
246086,West Bengal,PURULIA,2014,Summer,Rice,306.0,801.0
246087,West Bengal,PURULIA,2014,Summer,Sesamum,627.0,463.0
246088,West Bengal,PURULIA,2014,Whole Year,Sugarcane,324.0,16250.0
246089,West Bengal,PURULIA,2014,Winter,Rice,279151.0,597899.0


In [10]:
data = pd.read_csv(path, usecols=[col])
data

,Season
0,Kharif
1,Kharif
2,Kharif
3,Whole Year
4,Whole Year
...,...
246086,Summer
246087,Summer
246088,Whole Year
246089,Winter


In [16]:
type(data[col])

pandas.core.series.Series

In [18]:
col = "Season"
data = pd.read_csv(path, usecols = [col])
num = data[col].unique()
num

array(['Kharif     ', 'Whole Year ', 'Autumn     ', 'Rabi       ',
       'Summer     ', 'Winter     '], dtype=object)

In [7]:
crop = pd.read_csv(path)
crop.Season = crop.Season.apply(lambda s: s.rstrip())
crop.Crop = crop.Crop.apply(lambda s: s.rstrip())
crop.Crop.unique()

array(['Arecanut', 'Other Kharif pulses', 'Rice', 'Banana', 'Cashewnut',
       'Coconut', 'Dry ginger', 'Sugarcane', 'Sweet potato', 'Tapioca',
       'Black pepper', 'Dry chillies', 'other oilseeds', 'Turmeric',
       'Maize', 'Moong(Green Gram)', 'Urad', 'Arhar/Tur', 'Groundnut',
       'Sunflower', 'Bajra', 'Castor seed', 'Cotton(lint)', 'Horse-gram',
       'Jowar', 'Korra', 'Ragi', 'Tobacco', 'Gram', 'Wheat', 'Masoor',
       'Sesamum', 'Linseed', 'Safflower', 'Onion', 'other misc. pulses',
       'Samai', 'Small millets', 'Coriander', 'Potato',
       'Other  Rabi pulses', 'Soyabean', 'Beans & Mutter(Vegetable)',
       'Bhindi', 'Brinjal', 'Citrus Fruit', 'Cucumber', 'Grapes', 'Mango',
       'Orange', 'other fibres', 'Other Fresh Fruits', 'Other Vegetables',
       'Papaya', 'Pome Fruit', 'Tomato', 'Rapeseed &Mustard', 'Mesta',
       'Cowpea(Lobia)', 'Lemon', 'Pome Granet', 'Sapota', 'Cabbage',
       'Peas  (vegetable)', 'Niger seed', 'Bottle Gourd', 'Sannhamp',
       'Var

In [56]:
selects = dict(
    rice=['Rice','Paddy'],
    wheat=['Wheat'],
    oilseed=['Oilseeds total','other oilseeds'],
    cereal=['Small millets', 'Bajra', 'Other Cereals & Millets', 'Maize', 'Barley'],
    pulse=['Other Kharif pulses', 'other misc. pulses', 'Other  Rabi pulses', 'Peas & beans (Pulses)', 'Pulses total'],
    cotton=['Cotton(lint)']
)
for yr, annual_total in crop.groupby(crop.Crop_Year):
    save_dict = dict()
    for k, v in selects.items():
        if k == 2015:
            continue
        total = annual_total.loc[annual_total.Crop.isin(v)]
        production = total.Production.groupby(total.District_Name).sum() / 1e3
        area = total.Area.groupby(total.District_Name).sum() 
        save_dict[f'{k}_production'] = production.values
        save_dict[f'{k}_area'] = area.values
        save_dict[f'{k}_name'] = area.index.values
        
    sio.savemat(f'../Matlab/crop/district_crop_{yr}.mat', save_dict)